In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.listdir("../data")

['sample_submission.csv', 'test.csv', 'train.csv']

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")

In [4]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

 #TabNetRegressor()
#clf.fit(
#  X_train, Y_train,
#  eval_set=[(X_valid, y_valid)]
#)
#preds = clf.predict(X_test)

In [5]:
xcolumns = list(test.columns)
ycolumns = "NSP"

In [6]:
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

class Gini(Metric):
    def __init__(self):
        self._name = "gini"
        self._maximize = True

    def __call__(self, y_true, y_score):
        #print(y_true, np.argmax(y_score, 1))
        n_size = np.max(y_true)+1
        y_true_ = np.eye(n_size)[y_true]
        auc = roc_auc_score(y_true_, y_score, average='macro', multi_class="ovr")
        return max(2*auc - 1, 0.)

class F1(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True
        
    def __call__(self, y_true, y_score):
        return f1_score(y_true, np.argmax(y_score, 1), average='weighted')

In [7]:

X = train[xcolumns].values
y = train[ycolumns].values
X_test = test[xcolumns].values

In [8]:
from sklearn.preprocessing import QuantileTransformer
rng = np.random.RandomState(0)

transformer=QuantileTransformer(n_quantiles=300, output_distribution='normal')
X_t = transformer.fit_transform(X)
X_test_t = transformer.transform(X_test)

In [9]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

preds = []
cv_preds = []
for train_index, test_index in skf.split(X_t, y):
    xtrain = X_t[train_index]
    ytrain = y[train_index]
    xval = X_t[test_index]
    yval = y[test_index]
    clf = TabNetClassifier(seed=42) 
    clf.fit(
        xtrain, ytrain, eval_set=[(xval, yval)], weights=1,
        eval_metric=[F1, Gini]
    )
    cv_preds.append(clf.predict_proba(X_t))
    preds.append(clf.predict_proba(X_test_t))

Device used : cpu
epoch 0  | loss: 1.30106 | val_0_f1: 0.53125 | val_0_gini: 0.0     |  0:00:00s
epoch 1  | loss: 0.95786 | val_0_f1: 0.50825 | val_0_gini: 0.22083 |  0:00:00s
epoch 2  | loss: 0.8325  | val_0_f1: 0.43052 | val_0_gini: 0.22036 |  0:00:00s
epoch 3  | loss: 0.81245 | val_0_f1: 0.47104 | val_0_gini: 0.27272 |  0:00:01s
epoch 4  | loss: 0.74872 | val_0_f1: 0.5282  | val_0_gini: 0.29519 |  0:00:01s
epoch 5  | loss: 0.67444 | val_0_f1: 0.56938 | val_0_gini: 0.27906 |  0:00:01s
epoch 6  | loss: 0.62339 | val_0_f1: 0.61559 | val_0_gini: 0.34914 |  0:00:01s
epoch 7  | loss: 0.56238 | val_0_f1: 0.61952 | val_0_gini: 0.30265 |  0:00:02s
epoch 8  | loss: 0.55531 | val_0_f1: 0.62377 | val_0_gini: 0.30021 |  0:00:02s
epoch 9  | loss: 0.53973 | val_0_f1: 0.62523 | val_0_gini: 0.35278 |  0:00:02s
epoch 10 | loss: 0.50996 | val_0_f1: 0.6102  | val_0_gini: 0.32877 |  0:00:02s
epoch 11 | loss: 0.50354 | val_0_f1: 0.62309 | val_0_gini: 0.39261 |  0:00:03s
epoch 12 | loss: 0.46929 | val_0_f

epoch 48 | loss: 0.27765 | val_0_f1: 0.79155 | val_0_gini: 0.78083 |  0:00:12s
epoch 49 | loss: 0.26833 | val_0_f1: 0.80392 | val_0_gini: 0.80518 |  0:00:13s
epoch 50 | loss: 0.28506 | val_0_f1: 0.81426 | val_0_gini: 0.8145  |  0:00:13s
epoch 51 | loss: 0.27914 | val_0_f1: 0.81481 | val_0_gini: 0.79723 |  0:00:13s
epoch 52 | loss: 0.25943 | val_0_f1: 0.81598 | val_0_gini: 0.80352 |  0:00:14s
epoch 53 | loss: 0.2334  | val_0_f1: 0.82173 | val_0_gini: 0.8127  |  0:00:14s
epoch 54 | loss: 0.21765 | val_0_f1: 0.82004 | val_0_gini: 0.81119 |  0:00:14s
epoch 55 | loss: 0.26339 | val_0_f1: 0.82099 | val_0_gini: 0.80171 |  0:00:14s
epoch 56 | loss: 0.21394 | val_0_f1: 0.83086 | val_0_gini: 0.78962 |  0:00:15s
epoch 57 | loss: 0.24686 | val_0_f1: 0.82951 | val_0_gini: 0.77937 |  0:00:15s
epoch 58 | loss: 0.27549 | val_0_f1: 0.8264  | val_0_gini: 0.78827 |  0:00:15s
epoch 59 | loss: 0.24103 | val_0_f1: 0.82501 | val_0_gini: 0.80102 |  0:00:15s
epoch 60 | loss: 0.22208 | val_0_f1: 0.82846 | val_0

epoch 33 | loss: 0.30323 | val_0_f1: 0.77368 | val_0_gini: 0.81073 |  0:00:08s
epoch 34 | loss: 0.2844  | val_0_f1: 0.77311 | val_0_gini: 0.81181 |  0:00:09s
epoch 35 | loss: 0.30874 | val_0_f1: 0.77056 | val_0_gini: 0.80854 |  0:00:09s

Early stopping occured at epoch 35 with best_epoch = 25 and best_val_0_gini = 0.83924
Best weights from best epoch are automatically used!
Device used : cpu
epoch 0  | loss: 1.23754 | val_0_f1: 0.50728 | val_0_gini: 0.0     |  0:00:00s
epoch 1  | loss: 0.97797 | val_0_f1: 0.42361 | val_0_gini: 0.11104 |  0:00:00s
epoch 2  | loss: 0.90007 | val_0_f1: 0.39364 | val_0_gini: 0.09773 |  0:00:00s
epoch 3  | loss: 0.82504 | val_0_f1: 0.44272 | val_0_gini: 0.1764  |  0:00:00s
epoch 4  | loss: 0.76762 | val_0_f1: 0.5458  | val_0_gini: 0.31317 |  0:00:01s
epoch 5  | loss: 0.74242 | val_0_f1: 0.6314  | val_0_gini: 0.36599 |  0:00:01s
epoch 6  | loss: 0.69633 | val_0_f1: 0.6527  | val_0_gini: 0.44073 |  0:00:01s
epoch 7  | loss: 0.68723 | val_0_f1: 0.67319 | val_0

In [10]:
#!pip install --upgrade pytorch_tabnet
np.mean(np.stack(cv_preds).mean(0).argmax(1)+1==y)

0.8276470588235294

In [11]:
sample_submission['NSP'] = np.stack(preds).mean(0).argmax(1)+1

In [13]:
sample_submission.to_csv("prediction_tabnet.csv", index=None)